In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv();


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_qdrant import QdrantVectorStore
from langchain_core.output_parsers import StrOutputParser


In [3]:
pdf_path = Path(os.getcwd()).parent / "documents" / "doc_nodejs.pdf"
loader = PyPDFLoader(file_path=str(pdf_path))
docs = loader.load()


Ignoring wrong pointing object 268 0 (offset 0)
Ignoring wrong pointing object 309 0 (offset 0)


In [4]:
print((docs[0]))


page_content='A PDF Reference for        The Complete Node.js Dev Course                Version 3.0' metadata={'producer': 'macOS Version 10.14.1 (Build 18B75) Quartz PDFContext', 'creator': 'Acrobat PDFMaker 17 for Word', 'creationdate': "D:20190227140340Z00'00'", 'author': 'Andrew Mead', 'moddate': "D:20190227140340Z00'00'", 'source': 'd:\\Python\\gen_ai\\documents\\doc_nodejs.pdf', 'total_pages': 125, 'page': 0, 'page_label': '1'}


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
splitted_docs = splitter.split_documents(documents=docs)
print(len(splitted_docs))


333


In [ ]:
print(splitted_docs[0])


page_content='A PDF Reference for        The Complete Node.js Dev Course                Version 3.0' metadata={'producer': 'macOS Version 10.14.1 (Build 18B75) Quartz PDFContext', 'creator': 'Acrobat PDFMaker 17 for Word', 'creationdate': "D:20190227140340Z00'00'", 'author': 'Andrew Mead', 'moddate': "D:20190227140340Z00'00'", 'source': 'd:\\Python\\gen_ai\\documents\\doc_nodejs.pdf', 'total_pages': 125, 'page': 0, 'page_label': '1'}


In [ ]:
embedder = CohereEmbeddings(cohere_api_key=os.getenv("COHERE_API_KEY"), model="embed-english-v3.0")


In [ ]:
embedder.embed_query("Hey there")


[0.007106781,
 -0.0050621033,
 -0.029937744,
 -0.016235352,
 0.014663696,
 -0.022979736,
 0.006843567,
 0.010925293,
 0.032562256,
 0.018554688,
 -0.024124146,
 0.016708374,
 0.041931152,
 -0.051727295,
 0.07220459,
 -0.009269714,
 0.0052452087,
 0.05429077,
 -0.009597778,
 0.00014710426,
 -0.041748047,
 0.024154663,
 -0.046813965,
 0.03439331,
 0.03152466,
 -0.0017538071,
 0.00016081333,
 0.046020508,
 -0.017471313,
 -0.012008667,
 0.0024337769,
 0.00028038025,
 0.015106201,
 0.004890442,
 0.0008406639,
 0.026046753,
 0.028121948,
 -0.04916382,
 0.018035889,
 -0.011314392,
 0.031585693,
 -0.029464722,
 -0.06365967,
 -0.04586792,
 -0.0491333,
 0.02571106,
 -0.059661865,
 -0.029342651,
 0.07171631,
 -0.038391113,
 0.0017147064,
 0.005882263,
 0.008880615,
 -0.011421204,
 -0.03793335,
 -0.018081665,
 -0.019683838,
 -0.014923096,
 0.011779785,
 -0.0065689087,
 0.021316528,
 -0.0064849854,
 0.06384277,
 0.036590576,
 -0.024795532,
 -0.016708374,
 -0.006149292,
 0.0113220215,
 -0.015930176,

In [ ]:
vector_store = QdrantVectorStore.from_documents(
    documents=[],
    embedding=embedder, 
    url="http://localhost:6333",
    collection_name="qdrant_basic_rag"
)


In [ ]:
vector_store.add_documents(documents=splitted_docs)


['ede92bd0b1f94f95af133c3cb2e97cf6',
 'deaa2d4bf26a4e939ccf4ed301aeae89',
 'b06f62544260484fb25298104e17fc15',
 '3a1032c2f005461c8e4a0e5406f88c54',
 '8fd93adf490f4529bfdbfe39996815c1',
 'd86e33a9bada4584960697254460c51e',
 'ce2587f0ced84b6f8510a5e8fee4a543',
 '63aa6cb5db7d427c8c5550950bf4eb17',
 '6539b71697a34e90852b621374a8db2a',
 '1e13db5604a84fe8a102fb49c72a0a81',
 'b267de093ed14effa1b6a3776c67f4df',
 '5cef4536ddc54c74b342808a4cf820fe',
 '01f7ea29aa444820a1d6f1a5808930cf',
 '1043f8a8db1d42a9813169a11e72f278',
 '9cc7b6c6d9044305a34d3792b69e487c',
 'c8b7a47c7ab6419488ec1a36de40b550',
 '286100c77c6143ec85d73afe14485d56',
 'c9390e0e0d1440c7b49146324c53f00c',
 'de4cc3a2e2d649aa89de4665e25cf8f0',
 '4aa4d642cbb04d7395a5af487db7510c',
 '282c07b6eeb04671970b1c347963c485',
 '0d123b2506fc46fd9ae900304889869b',
 '2c0dce9ba24c4387a7884a7a7c8dabe2',
 '445e1d823feb433f9607ff60337fe229',
 'e3848103f5434c66a4dd154bb61b93dd',
 'fc26662c3cc44b62b64302165b48a3e9',
 'a9238c834e674415af62ab6938833554',
 

In [ ]:
retriever = QdrantVectorStore.from_existing_collection(
    collection_name="qdrant_basic_rag",
    url="http://localhost:6333",
    embedding=embedder
)


In [ ]:
llm = ChatGoogleGenerativeAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini-1.5-flash"
)


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant who respond to the query only from the given context.
     
     <context>
     {context}
     </context>
     
     """),
    ("human", "{query}")
])


In [ ]:
chain = prompt | llm | StrOutputParser()


In [ ]:
query = input(">>>>>>")

result = retriever.similarity_search(query)
content = []
for doc in result:
    content.append(doc.page_content)

# print(content)

output = chain.invoke(input={
    "query": query,
    "context": content
})
print(output)


Based on the provided text, the `fs` module is a built-in Node.js module that provides functions for manipulating the file system.  The example shows it being used with `writeFileSync` to write data to a file.
